In [1]:
%%time
%load_ext autoreload
%autoreload 2

# Standard Python modules
import time 
import numpy as np
import os
import sys
import ipdb
import requests

sys.path.append(os.path.abspath(".."))
import json
import re
import glob
import asyncio
import aiohttp
import openai
from typing import Dict, Tuple, List, Callable
from dotenv import load_dotenv
from pathlib import Path
import PyPDF2

# Project imports
from src.utils import (timeit, encode_image, plt, pylab, find_bad_pagenos, 
                    delete_bad_pagenos, dump_fragmented_output_to_json, get_corresponding_bad_fnames,
                    load_output_from_json, logging_for_main, dump_output_to_json, log_execution_time, count_num_tokens)
from src.processing import compute_log_spectrum_1d, extract_image_bbox, save_images
from src.api_requests import construct_payload_for_gpt, process_single_page
from src.document_generation import save_document, setup_logger, chapter_splitter

# Display and plotting, set notebook display width
from IPython.display import display, HTML, clear_output

display(HTML("<style>.container { width:90% !important; }</style>"))

# Print Python environment info
print('sys.executable:', sys.executable)
print('sys.version:', sys.version, '\n')

# Load environment variables from .env file
env_path = Path('../.env')  # Adjust path if needed
load_dotenv(dotenv_path=env_path)

# OpenAI API setup
openai.api_key = os.getenv("OPENAI_API_KEY")

# Setup for PDF processing
foldername = "Der Weltkrieg v2"

# # split the pdf into separate pages
chapter_splitter(input_pdf_path=f"../input_data/{foldername} East Front.pdf", 
                 output_folder=f"../input_data/{foldername}")

chapter_splitter - L31 - INFO - Extracted 381 pages from PDF


sys.executable: /Users/ozkansafak/code/fraktur/.venv/bin/python3
sys.version: 3.10.9 (main, Mar  1 2023, 12:20:14) [Clang 14.0.6 ] 



chapter_splitter - L45 - INFO - All pages have been split and saved to ../input_data/Der Weltkrieg v2


CPU times: user 1.55 s, sys: 2.32 s, total: 3.87 s
Wall time: 1.21 s


In [2]:
# Initialize variables
plotter = False
image_path = f"../input_data/{foldername}/*pdf"
fnames = sorted(glob.glob(image_path))
all_pagenos = [re.search(r'page_(.*?)\.pdf', fname, re.DOTALL).group(1) for fname in fnames]

# Storage for processed texts
raw_german_texts: Dict[str, str] = {}
german_texts: Dict[str, str] = {}
english_texts: Dict[str, str] = {}

# Configure logging
logger = setup_logger('time_logger')

---
## PART 1
## Fraktur Translator (GPT-4o)

In [3]:
# raw_german_texts, german_texts, english_texts, _ = load_output_from_json(foldername) 

In [4]:
@log_execution_time
async def main(fnames, model_name="", semaphore_count=10, extract=True):
    semaphore = asyncio.Semaphore(semaphore_count)  # Adjust number based on API limits
    async def wrapper_process_page(fname: str) -> Tuple[str, Dict]:
        global raw_german_texts, german_texts, english_texts  # Explicitly declare globals
        async with semaphore:
            pageno = re.search(r'page_(.*?)\.pdf', fname, re.DOTALL).group(1)
            result = await process_single_page(fname, model_name, plotter, pageno, extract) 
            return pageno, result
    
    # A list of coroutine objects. include only the unprocessed pages.
    tasks = []
    print(f'Composing tasks to be executed with asyncio, semaphore_count = {semaphore_count} ')
    for fname in fnames:
        pageno = re.search(r'page_(.*?)\.pdf', fname, re.DOTALL).group(1)
        if pageno not in set(raw_german_texts.keys()):
            print(f'{pageno} ', end='')
            tasks.append(wrapper_process_page(fname))
    print(f"main: len(tasks): {len(tasks)} -- Processing tasks as they complete")
    
    # Process tasks as they complete
    for i, task in enumerate(asyncio.as_completed(tasks)):
        try:
            pageno, (content, token_count, raw_german_text, german_text, english_text) = await task
            raw_german_texts[pageno] = raw_german_text
            german_texts[pageno] = german_text
            english_texts[pageno] = english_text
            
            logging_for_main(i, tasks, pageno, token_count, raw_german_text, german_text, english_text)
        except Exception as e:
            logger.error(f"{i} of {len(tasks)-1} - Error processing a task: {e}. token_count:{token_count}")
    return 

# Run the async code
await main(fnames, model_name="gpt-4o-2024-08-06", semaphore_count=10)


Composing tasks to be executed with asyncio, semaphore_count = 10 
001 002 003 004 005 006 007 008 009 010 011 012 013 014 015 016 017 018 019 020 021 022 023 024 025 026 027 028 029 030 031 032 033 034 035 036 037 038 039 040 041 042 043 044 045 046 047 048 049 050 051 052 053 054 055 056 057 058 059 060 061 062 063 064 065 066 067 068 069 070 071 072 073 074 075 076 077 078 079 080 081 082 083 084 085 086 087 088 089 090 091 092 093 094 095 096 097 098 099 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 2

logging_for_main - L251 - INFO - 0 of 380 -- Successfully processed pageno:331. token_count:1804
logging_for_main - L251 - INFO - 1 of 380 -- Successfully processed pageno:239. token_count:1930
logging_for_main - L251 - INFO - 2 of 380 -- Successfully processed pageno:330. token_count:1706
logging_for_main - L251 - INFO - 3 of 380 -- Successfully processed pageno:134. token_count:1831
logging_for_main - L251 - INFO - 4 of 380 -- Successfully processed pageno:133. token_count:1895
logging_for_main - L251 - INFO - 5 of 380 -- Successfully processed pageno:327. token_count:1916
logging_for_main - L251 - INFO - 6 of 380 -- Successfully processed pageno:019. token_count:1921
logging_for_main - L251 - INFO - 7 of 380 -- Successfully processed pageno:241. token_count:2045
logging_for_main - L251 - INFO - 8 of 380 -- Successfully processed pageno:018. token_count:2051
logging_for_main - L251 - INFO - 9 of 380 -- Successfully processed pageno:243. token_count:140
logging_for_main - L251 - INFO 

pageno:247[this involved 2½ divisions of Landsturm²)] as a special army group near Krakow. This group was to cross the border on August 17 and advance shoulder to shoulder with the German Landwehr Corps against the Vistula north of the San estuary.
For the main forces to be assembled between Stanislau and the San estuary, it was stated: "From this assembly, the offensive will begin — depending on the deployment and behavior of the Russian forces — in an easterly direction (in the unlikely event that the mass of the Russians marched against the Galician eastern border) on the 20th, in a northerly direction on August 22 from the left flank. To support this offensive of the Austro-Hungarian army, it is desired in all cases that the offensive of the 4½ army corps of Generaloberst v. Prittwitz proceed in the direction over Siedlce." ... — That was much further than the 1909 promoted offensive "against the Narew"³).
General v. Conrad repeated this new demand in a letter dated August 15⁴) and

logging_for_main - L251 - INFO - 221 of 380 -- Successfully processed pageno:261. token_count:1963
logging_for_main - L251 - INFO - 222 of 380 -- Successfully processed pageno:206. token_count:685
logging_for_main - L251 - INFO - 223 of 380 -- Successfully processed pageno:352. token_count:3245
logging_for_main - L251 - INFO - 224 of 380 -- Successfully processed pageno:013. token_count:2026
logging_for_main - L251 - INFO - 225 of 380 -- Successfully processed pageno:104. token_count:1495
logging_for_main - L251 - INFO - 226 of 380 -- Successfully processed pageno:358. token_count:2896
logging_for_main - L251 - INFO - 227 of 380 -- Successfully processed pageno:258. token_count:1970
logging_for_main - L251 - INFO - 228 of 380 -- Successfully processed pageno:249. token_count:1759
extract_text_section - L241 - ERROR - pageno: 205, section name:"raw_german" was not found. token_count=5000
extract_text_section - L241 - ERROR - pageno: 205, section name:"german" was not found. token_count=

pageno:246[Landwehr Corps1) 2)] against Tschenstochau and Kalisch. — Further 5 Reserve Divisions [Replacement Divisions1) 3)] would be available by the 11th day of mobilization and then deployed according to the situation. — The commander-in-chief has the task of binding as many enemy forces as possible and thereby facilitating victory for the Austro-Hungarian army. An early, strongly superior offensive by the Russians against East Prussia would, as further explained, in itself bring about the desired result. If such a result does not occur, the German Eastern Army will advance into Russia in a direction that brings the greatest relief to the Austrian army.” — The corps prepared against Tschenstochau—Kalisch is to carry out such an operation under all circumstances, to begin on the 12th day of mobilization (= August 13) and to march relentlessly over Radom against the Vistula above Iwangrod. — “The Austrian army can thus definitely count on active support for its offensive into Russia 

logging_for_main - L251 - INFO - 298 of 380 -- Successfully processed pageno:199. token_count:1915
logging_for_main - L251 - INFO - 299 of 380 -- Successfully processed pageno:251. token_count:1788
logging_for_main - L251 - INFO - 300 of 380 -- Successfully processed pageno:281. token_count:2069
logging_for_main - L251 - INFO - 301 of 380 -- Successfully processed pageno:224. token_count:1870
logging_for_main - L251 - INFO - 302 of 380 -- Successfully processed pageno:108. token_count:2135
logging_for_main - L251 - INFO - 303 of 380 -- Successfully processed pageno:168. token_count:1959
logging_for_main - L251 - INFO - 304 of 380 -- Successfully processed pageno:248. token_count:2287
logging_for_main - L251 - INFO - 305 of 380 -- Successfully processed pageno:031. token_count:1427
logging_for_main - L251 - INFO - 306 of 380 -- Successfully processed pageno:221. token_count:1861
logging_for_main - L251 - INFO - 307 of 380 -- Successfully processed pageno:030. token_count:463
logging_for

In [8]:
## V5
good_pagenos = {}
good_pagenos['Der Weltkrieg v5'] = {'213', '225', '221'}
good_pagenos['Der Weltkrieg v6'] = {'377', '378', '379'}

## V7 ('417' and '429' does not get OCRed by GPT-4o, and it gets summarized by Claude)
good_pagenos['Der Weltkrieg v7'] = {'265', '407', '409', '410', '412', '418', '420', '421', '423', '424', '428'} 

## V8 looks fine
good_pagenos['Der Weltkrieg v8'] = {'064', '369'}

## v10
good_pagenos['Der Weltkrieg v10'] = {'083'}
good_pagenos['Der Weltkrieg v11'] = {}
good_pagenos['Der Weltkrieg v2'] = {'246','247'}

bad_pagenos = find_bad_pagenos(all_pagenos, raw_german_texts, german_texts, english_texts, good_pagenos[foldername])
# delete_bad_pagenos(bad_pagenos, raw_german_texts, german_texts, english_texts)


> pageno: 006. `raw_german_texts['006']`: pageno: 006, "<raw_german>" section was not found

> pageno: 007. `raw_german_texts['007']`: pageno: 007, "<raw_german>" section was not found

> pageno: 008. `raw_german_texts['008']`: pageno: 008, "<raw_german>" section was not found

> pageno: 023. `raw_german_texts['023']`: pageno: 023, "<raw_german>" section was not found

> pageno: 166. `german_texts['166']`: pageno: 166, "<german>" section was not found

> pageno: 169. `english_texts['169']`: pageno: 169, "<english>" section was not found

> pageno: 205. `raw_german_texts['205']`: pageno: 205, "<raw_german>" section was not found

> pageno: 238. `raw_german_texts['238']`: pageno: 238, "<raw_german>" section was not found

> pageno: 360. `raw_german_texts['360']`: pageno: 360, "<raw_german>" section was not found

bad_pagenos (9): ['006', '007', '008', '023', '166', '169', '205', '238', '360']


---
## PART 2
##  Handle missing keys (Claude Sonnet)

In [9]:
# Check for unprocessed pages and print them to stdout.
# ------------------------------------------------------------------------------------
# 1. Rerun the missing pages on Claude without performing FFT based extraction. 
# ------------------------------------------------------------------------------------

# Get missing keys based on empty raw_german_texts
bad_pagenos = find_bad_pagenos(all_pagenos, raw_german_texts, german_texts, english_texts, good_pagenos[foldername])
delete_bad_pagenos(bad_pagenos, raw_german_texts, german_texts, english_texts)
await main(fnames, model_name="claude-3-5-sonnet-20241022", semaphore_count=1, extract=False)

# ------------------------------------------------------------------------------------
# 2. If there still are missing pages, run them performing FFT based extraction. 
#    This time compute missing_keys based on 'english_texts'.
# ------------------------------------------------------------------------------------

# Now get missing keys based on empty english_texts
bad_pagenos = find_bad_pagenos(all_pagenos, raw_german_texts, german_texts, english_texts, good_pagenos[foldername])
delete_bad_pagenos(bad_pagenos, raw_german_texts, german_texts, english_texts)
await main(fnames, model_name="claude-3-5-sonnet-20241022", semaphore_count=1, extract=True)

bad_pagenos = find_bad_pagenos(all_pagenos, raw_german_texts, german_texts, english_texts, good_pagenos[foldername])
delete_bad_pagenos(bad_pagenos, raw_german_texts, german_texts, english_texts)
await main(fnames, model_name="claude-3-5-sonnet-20241022", semaphore_count=1, extract=False)

bad_pagenos = find_bad_pagenos(all_pagenos, raw_german_texts, german_texts, english_texts, good_pagenos[foldername])



> pageno: 006. `raw_german_texts['006']`: pageno: 006, "<raw_german>" section was not found

> pageno: 007. `raw_german_texts['007']`: pageno: 007, "<raw_german>" section was not found

> pageno: 008. `raw_german_texts['008']`: pageno: 008, "<raw_german>" section was not found

> pageno: 023. `raw_german_texts['023']`: pageno: 023, "<raw_german>" section was not found

> pageno: 166. `german_texts['166']`: pageno: 166, "<german>" section was not found

> pageno: 169. `english_texts['169']`: pageno: 169, "<english>" section was not found

> pageno: 205. `raw_german_texts['205']`: pageno: 205, "<raw_german>" section was not found

> pageno: 238. `raw_german_texts['238']`: pageno: 238, "<raw_german>" section was not found

> pageno: 360. `raw_german_texts['360']`: pageno: 360, "<raw_german>" section was not found

bad_pagenos (9): ['006', '007', '008', '023', '166', '169', '205', '238', '360']
------------------------------------------------------------------
> Deleting pageno: 006. Plea

logging_for_main - L251 - INFO - 0 of 8 -- Successfully processed pageno:205. token_count:1961
logging_for_main - L251 - INFO - 1 of 8 -- Successfully processed pageno:007. token_count:2367
logging_for_main - L248 - ERROR - main pageno:008. '[' present in german_text. token_count:1591


pageno:008[Content continues with all maps and sketches as in raw_german]
</body>

------------------------------------------------------------------


logging_for_main - L251 - INFO - 3 of 8 -- Successfully processed pageno:166. token_count:1897
logging_for_main - L251 - INFO - 4 of 8 -- Successfully processed pageno:360. token_count:1440
logging_for_main - L251 - INFO - 5 of 8 -- Successfully processed pageno:006. token_count:2900
logging_for_main - L251 - INFO - 6 of 8 -- Successfully processed pageno:169. token_count:2022
logging_for_main - L251 - INFO - 7 of 8 -- Successfully processed pageno:023. token_count:1833
logging_for_main - L251 - INFO - 8 of 8 -- Successfully processed pageno:238. token_count:1744
wrapper - L88 - INFO - Finished main in 316.14 seconds.
wrapper - L88 - INFO - Finished main in 0.01 seconds.
wrapper - L88 - INFO - Finished main in 0.00 seconds.



bad_pagenos (0): []
------------------------------------------------------------------
Composing tasks to be executed with asyncio, semaphore_count = 1 
main: len(tasks): 0 -- Processing tasks as they complete

bad_pagenos (0): []
------------------------------------------------------------------
Composing tasks to be executed with asyncio, semaphore_count = 1 
main: len(tasks): 0 -- Processing tasks as they complete

bad_pagenos (0): []


In [10]:
# save json outputs
dump_output_to_json(foldername, raw_german_texts, german_texts, english_texts)

# Save json files and .docx files.
doc1, fname1 = save_document(german_texts, foldername, language=f'{foldername} - German')
doc2, fname2 = save_document(english_texts, foldername, language=f'{foldername} - English')


../output_data/Der Weltkrieg v2 created

dumped to ../output_data/Der Weltkrieg v2/*json files.


``` 
1.  Upload Input folder of pdfs to blob storage.
2.  Read file from s3.
3.  FFT in y -> (x_hi, x_lo), write half_cropped_image to s3
4.  FFT in x -> (y_hi, y_lo), write cropped_image to s3
5.  Read cropped image from s3 -> encode_image -> translate and transcribe -> JSON output

```

### Available models and pricing:
```
"gpt-4o-2024-08-06":
    "price_txt": "$2.50 / 1M input tokens"
    "price_img": "$0.001913 / 1500px^2"
    
"gpt-4o-mini-2024-07-18":
    "price_txt": "$0.150 / 1M input tokens"
    "price_img": "$0.003825 / 1500px^2"
    
```

---
## PART 3
## Load the German text and translate broken sentences.

In [10]:
# initialize output dicts
english_texts_defragmented = {}
contents = {}
payloads = {}
fragments_2 = {None:''}

raw_german_texts, german_texts, english_texts, english_texts_defragmented = load_output_from_json(foldername, load_defrag=True)

In [12]:
from src.constants import FRAGMENTED_SENTENCES_PROMPT

def construct_payload_fragmented_sentences(
                        german_page_1: str,
                        german_page_2: str,
                        english_page_1_old: str,
                        german_page_1_top_fragment_to_be_ignored: str):

    model_name = "gpt-4o-2024-08-06"
    payload = {
        "model": model_name,
        "messages": [
          {
            "role": "system", 
            "content": "You are a World War II historian, who's bilingual in German and English "
              "You speak both languages with masterful efficiency and you're a professional translator from GERMAN to ENGLISH who "
              "stays loyal to both the style and the character of the original German text in your book translations."
          },
          {
            "role": "user",
            "content": [
              {
                  "type": "text",
                  "text": FRAGMENTED_SENTENCES_PROMPT.format(
                      german_page_1=german_page_1,
                      german_page_2=german_page_2,
                      english_page_1_old=english_page_1_old,
                      german_page_1_top_fragment_to_be_ignored=german_page_1_top_fragment_to_be_ignored
                  )
              },
            ]
          },
        ],
        "max_tokens": 5000,
        "temperature": 0.1
    }

    return payload 

In [13]:
from src.utils import log_execution_time_synchronous

def make_gpt_request_for_broken_sentences(payload: dict, pageno: str) -> dict:
    global english_texts_defragmented 

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {openai.api_key}"
    }

    model_name = "gpt-4o-2024-08-06"
    response = requests.post(
        "https://api.openai.com/v1/chat/completions",
        json=payload,
        headers=headers
    )
    result = response.json()
    try:
        content = result['choices'][0]['message']['content']
        english_texts_defragmented[pageno] = re.search(r'<english_page_1_new>(.*?)</english_page_1_new>', 
                                                       content, re.DOTALL).group(1)
        return content
    except Exception as e:
        print(f"hey An Error occurred. Error details: {e}")
        ipdb.set_trace()
        return None

@log_execution_time_synchronous
def main_broken_sentences():
    global payloads, contents, fragments_2
    for i in range(len(all_pagenos) - 1):
        pageno = all_pagenos[i]
        prev_pageno = all_pagenos[i-1] if i > 0 else None
        next_pageno = all_pagenos[i+1]
        if pageno in english_texts_defragmented.keys() and len(english_texts_defragmented[pageno]) > 10:
            logger.info(f'pageno:{pageno} already in `english_texts_defragmented`... skipping')
            continue
        
        for trial in [1,2,3]:
            try:
                logger.info(f"Processing pageno: {pageno} {' ...trying again '+ str(trial) if trial > 1 else ''}")
                time.sleep(.1)
                payload = construct_payload_fragmented_sentences(
                    german_texts[pageno],
                    german_texts[next_pageno],
                    english_texts[pageno],
                    fragments_2[prev_pageno]
                )
                payloads[pageno] = payload['messages'][1]['content'][0]['text']
                content = make_gpt_request_for_broken_sentences(payload, pageno)
                contents[pageno] = content
                fragments_2[pageno] = re.search(r'<fragment_2>(.*?)</fragment_2>', content, re.DOTALL).group(1)
                if fragments_2[pageno].count('\n') > 10:
                    print('not accepting fragment_2. fragments_2[pageno].count("\\n")', fragments_2[pageno].count('\n'))
                    fragments_2[pageno] = '' 
                logger.info(f"pageno:{pageno} next round's fragment_2: {fragments_2[pageno]}") 
                break
            except Exception as e:
                logger.error(f"Error processing. {e}. trial:{trial}, pageno: {pageno}")
                if trial == 3:
                    english_texts_defragmented[pageno] = ''
        
        if pageno[-1] == '0' or pageno == all_pagenos[-2]:
            dump_fragmented_output_to_json(foldername, english_texts_defragmented, fragments_2, contents)
            
    english_texts_defragmented[next_pageno] = ''

main_broken_sentences()


main_broken_sentences - L41 - INFO - Processing pageno: 001 
main_broken_sentences - L56 - INFO - pageno:001 next round's fragment_2: Mit vierzehn Karten und elf Skizzen
*
EML
Verlegt bei E. S. Mittler & Sohn
Berlin im Jahre 1925
main_broken_sentences - L41 - INFO - Processing pageno: 002 
main_broken_sentences - L56 - INFO - pageno:002 next round's fragment_2: 
main_broken_sentences - L41 - INFO - Processing pageno: 003 
main_broken_sentences - L56 - INFO - pageno:003 next round's fragment_2: Die Historische Kommission hat Herrn General der Infanterie a. D. Dr. phil. h. c. Hugo Freiherrn v. Freytag-Loringhoven in Weimar sowie Herrn Geheimen Hofrat Dr. Walter Goetz, ordentlichen Professor der Geschichte an der Universität Leipzig, M. d. R., mit der Prüfung des zweiten Bandes beauftragt.
main_broken_sentences - L41 - INFO - Processing pageno: 004 
main_broken_sentences - L56 - INFO - pageno:004 next round's fragment_2: 
main_broken_sentences - L41 - INFO - Processing pageno: 005 
main_b

dumped fragmented outputs foldername: Der Weltkrieg v2.
max(keys) = 010


main_broken_sentences - L56 - INFO - pageno:011 next round's fragment_2: Chef des österreichisch-ungarischen Generalstabes, seit 1906 General der Infanterie Freiherr Conrad v. Hötzendorf, trat daraufhin am 1. Januar 1909 an General v. Moltke heran, um zu klären, welche Maßnahmen sich für beide Mächte aus einem militärischen Eingreifen Rußlands gegen die Donau-Monarchie ergeben könnten.
main_broken_sentences - L41 - INFO - Processing pageno: 012 
main_broken_sentences - L56 - INFO - pageno:012 next round's fragment_2: mit Serbien sich ruhig verhalten wird, scheint mir durchaus nicht unwahr- scheinlich.
main_broken_sentences - L41 - INFO - Processing pageno: 013 
main_broken_sentences - L56 - INFO - pageno:013 next round's fragment_2: zusammenzuwirken, könne dann „rein räumlich theoretisch genommen“ sogar zur Frage stehen, den österreichisch-ungarischen Aufmarsch unter Besetzung der Karpatenpässe bis in die Linie Bartfeld—Neu-Sandez—Bochnia (35 km östlich Krakau) — das hieß also weit nac

dumped fragmented outputs foldername: Der Weltkrieg v2.
max(keys) = 020


main_broken_sentences - L56 - INFO - pageno:021 next round's fragment_2: Die Rüstungen und Stärkeverhältnisse bis zum Sommer 1914.
main_broken_sentences - L41 - INFO - Processing pageno: 022 
main_broken_sentences - L56 - INFO - pageno:022 next round's fragment_2: in einigen Punktten über die Kriegsbereitschaft der übrigen Großmächte einschließlich Deutschlands erhebt, nämlich: durch das Ausscheiden der militärischen Schwächeperiode im Winter infolge Zurückbehaltung des ältesten Jahrgangs bis zur vollendeten Ausbildung der Rekruten, — durch häufige praktische Erprobung der gesamten Mobilmachungsmaßnahmen mit Hilfe der Versuchs- und Probemobilmachungen, — durch die Möglichkeit außerordentlicher Beschleunigung der Mobilmachung mit Hilfe der Kriegsvorbereitungsperiode
main_broken_sentences - L41 - INFO - Processing pageno: 023 
main_broken_sentences - L56 - INFO - pageno:023 next round's fragment_2: In bezug auf Ausbildung, Bewaffnung und Ausrüstung wurde das russische Heer als vollwertig

...

dumped fragmented outputs foldername: Der Weltkrieg v2.
max(keys) = 360


main_broken_sentences - L56 - INFO - pageno:361 next round's fragment_2: 
main_broken_sentences - L41 - INFO - Processing pageno: 362 
main_broken_sentences - L56 - INFO - pageno:362 next round's fragment_2: 
main_broken_sentences - L41 - INFO - Processing pageno: 363 
main_broken_sentences - L56 - INFO - pageno:363 next round's fragment_2: 
main_broken_sentences - L41 - INFO - Processing pageno: 364 
main_broken_sentences - L56 - INFO - pageno:364 next round's fragment_2: 1. Don-Kosaken-Division
main_broken_sentences - L41 - INFO - Processing pageno: 365 
main_broken_sentences - L56 - INFO - pageno:365 next round's fragment_2: 11., 12., 13. u. 14. sibir. Schützen-Division, 1. u. 2. taufas., 6. turkestan. Schützen-Brigade
main_broken_sentences - L41 - INFO - Processing pageno: 366 
main_broken_sentences - L56 - INFO - pageno:366 next round's fragment_2: 
main_broken_sentences - L41 - INFO - Processing pageno: 367 
main_broken_sentences - L56 - INFO - pageno:367 next round's fragment_2:

dumped fragmented outputs foldername: Der Weltkrieg v2.
max(keys) = 370


main_broken_sentences - L56 - INFO - pageno:371 next round's fragment_2: 
main_broken_sentences - L41 - INFO - Processing pageno: 372 
main_broken_sentences - L56 - INFO - pageno:372 next round's fragment_2: 
main_broken_sentences - L41 - INFO - Processing pageno: 373 
main_broken_sentences - L56 - INFO - pageno:373 next round's fragment_2: 
main_broken_sentences - L41 - INFO - Processing pageno: 374 
main_broken_sentences - L56 - INFO - pageno:374 next round's fragment_2: 
main_broken_sentences - L41 - INFO - Processing pageno: 375 
main_broken_sentences - L56 - INFO - pageno:375 next round's fragment_2: 
main_broken_sentences - L41 - INFO - Processing pageno: 376 
main_broken_sentences - L56 - INFO - pageno:376 next round's fragment_2: 2. Kav. Brig. 89.
main_broken_sentences - L41 - INFO - Processing pageno: 377 
main_broken_sentences - L56 - INFO - pageno:377 next round's fragment_2: 
main_broken_sentences - L41 - INFO - Processing pageno: 378 
main_broken_sentences - L56 - INFO - p

dumped fragmented outputs foldername: Der Weltkrieg v2.
max(keys) = 380


In [11]:
# dump_fragmented_output_to_json(foldername, english_texts_defragmented, fragments_2, contents)

doc1, fname1 = save_document(german_texts, foldername, language=f'{foldername} - German_defragmented')
doc2, fname2 = save_document(english_texts_defragmented, foldername, language=f'{foldername} - English_defragmented')


---

## PART 4
## Fix Broken Paragraphs

In [12]:
from src.constants import *
# print(CARRIAGE_RETURN_SYSTEM_PROMPT)
# print()
# print(CARRIAGE_RETURN_USER_PROMPT)

def construct_payload_for_gpt_slash_n(english_text_defragmented) -> dict:
    model_name = "gpt-4o-2024-08-06"
    payload = {
        "model": model_name,
        "messages": [
          {
            "role": "system", 
            "content": CARRIAGE_RETURN_SYSTEM_PROMPT
          },
          {
            "role": "user",
            "content": [
              {
                "type": "text",
                "text": CARRIAGE_RETURN_USER_PROMPT.format(
                    english_text_defragmented=english_text_defragmented
                )},
            ]
          }
        ],
        "max_tokens": 5000,
        "temperature": 0.1
    }
    return payload


In [13]:
problematic = {}
for pageno in all_pagenos:
    text = english_texts_defragmented[pageno]
    
    n_count = text.count('\n')
    if n_count > 15:
        problematic[pageno] = text

print('len(problematic)', len(problematic))

for pageno in problematic:
    print('pageno:', pageno)
    print(problematic[pageno])

len(problematic) 96
pageno: 005

<header>Table of Contents</header>
<body>
Introduction to the Second Volume ................................................. IX
Preliminary Remarks ................................................................. XV
First Part.
The Campaign Plan Against Russia and the Deployment in the East.
First Chapter. The Agreements with Austria-Hungary ................ 3
Second Chapter. The Armaments and Strength Ratios up to
Summer 1914 ................................................................. 15
Third Chapter. The Austro-Hungarian Deployment Plan and the
Start of the War ................................................................. 24
Fourth Chapter. The Russian Deployment .................................. 32
Second Part.
The Operations in East Prussia under Generaloberst v. Prittwitz.
First Chapter. Until the Start of Operations.
1. The Defense of the German East .................................. 39
2. The German 8th Army .......................

In [14]:
async def make_gpt_request_slash_n(text) -> dict:
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {openai.api_key}"
    }

    async with aiohttp.ClientSession() as session:
        async with session.post(
            "https://api.openai.com/v1/chat/completions",
            json=construct_payload_for_gpt_slash_n(text),
            headers=headers
        ) as response:
            return await response.json()


In [15]:
@log_execution_time
async def main_slash_n(model_name="", semaphore_count=10):
    semaphore = asyncio.Semaphore(semaphore_count) 
    async def wrapper_process_page(problematic, pageno: str) -> Tuple[str, Dict]:
        async with semaphore:
            response = await make_gpt_request_slash_n(problematic[pageno]) 
            return pageno, response

    tasks = []
    print(f'Composing tasks to be executed with asyncio, semaphore_count = {semaphore_count}')
    for q, pageno in enumerate(problematic):
        print(f'{pageno} ', end='')
        tasks.append(wrapper_process_page(problematic, pageno))
    print(f"main: len(tasks): {len(tasks)} -- Processing tasks as they complete")

    responses = []
    for i, task in enumerate(asyncio.as_completed(tasks)):
        try:
            pageno, response = await task
            responses.append((pageno, response))
        except Exception as e:
            logger.error(f"{i} of {len(tasks)-1} - Error processing a task: {e}.")
    return responses

# Run the async code
responses = await main_slash_n(model_name="gpt-4o-2024-08-06")


Composing tasks to be executed with asyncio, semaphore_count = 10
005 006 007 008 013 014 019 020 023 028 036 038 052 057 070 075 084 085 089 091 094 101 108 111 114 125 129 130 132 143 149 151 152 159 171 183 191 200 201 211 213 226 234 240 241 242 244 247 248 250 258 262 265 277 279 295 298 301 304 313 323 331 338 348 349 350 351 352 353 354 355 356 357 358 359 360 361 362 363 364 365 366 367 368 369 370 371 372 373 374 375 376 377 378 379 380 main: len(tasks): 96 -- Processing tasks as they complete


wrapper - L88 - INFO - Finished main_slash_n in 54.07 seconds.


In [16]:
good_pagenos = {}
good_pagenos['Der Weltkrieg v8'] = {'176'}
good_pagenos['Der Weltkrieg v7'] = {''}
good_pagenos['Der Weltkrieg v2'] = {''}


In [17]:
from IPython.display import display, HTML
english_texts_fixed_paragraphs = {key: value for key, value in english_texts_defragmented.items()}

for pageno, response in sorted(responses):
    if pageno in good_pagenos[foldername]:
        continue
    result = response['choices'][0]['message']['content']
    match = re.search(r'<output_text>(.*?)</output_text>', result, re.DOTALL)
    if match:
        result = match.group(1).strip()  # Extract content and strip any leading/trailing whitespace

    print('---' * 22)
    print(f"pageno: {pageno}")
    if (len(result) < 40 and 'The input text is' in result and 'correct' in result):
        print(f'result: {result}')
    else:
        print(f'\n{english_texts_defragmented[pageno]}')
        print('result: ')
        display(HTML(f'<b style="color:blue;">{result}</b>'))
        english_texts_fixed_paragraphs[pageno] = result


------------------------------------------------------------------
pageno: 005
result: The input text is correctly OCR'ed.
------------------------------------------------------------------
pageno: 006
result: The input text is correctly OCR'ed.
------------------------------------------------------------------
pageno: 007
result: The input text is correctly OCR'ed.
------------------------------------------------------------------
pageno: 008
result: The input text is correctly OCR'ed.
------------------------------------------------------------------
pageno: 013


<pageno>6</pageno>
<header>The Agreements with Austria-Hungary.</header>
<body>that Serbia will behave calmly seems to me quite likely." However, if "despite everything" a great war comes,
Germany would be forced to first strike against France,
against which General v. Moltke, like Count Schlieffen, considered a
quick decision possible. He stated that through such a
decision, effective assistance would ultimately be provide

------------------------------------------------------------------
pageno: 014

to cooperate, then "purely spatially theoretically taken" could even 
be in question, to move back the Austro-Hungarian deployment under occupation 
of the Carpathian passes to the line Bartfeld—Neu-Sandez—Bochnia 
(35 km east of Krakow) — that meant far into West Galicia. However, there are great concerns about this. On the other hand, a deployment 
in the line Rudki (40 km southwest of Lemberg)—San estuary might be advantageous, 
to proceed from there to the offensive as soon as the Russian 
forces have reached the area of impact. General v. Conrad ties 
the question of the strength and behavior of the German forces remaining in the east 
in a two-front war, and whether they would be able to "bind 19½ Russian divisions." 
Furthermore, he asked about the strength, arrival time, and use of the reinforcements 
expected later from the western theater of war. 
General v. Moltke replied on February 24, 1909, th

------------------------------------------------------------------
pageno: 019


<pageno>12</pageno>
<header>The Agreements with Austria-Hungary.</header>
<body>Forces carry more than ever a danger within them, and continued: "If 
Austria needs all its forces to carry out the fight against Russia, the same 
applies to Germany in the fight against France. I would therefore advocate 
deploying our troops prepared in the East in the West as well, if 
consideration for Austria did not prevent me from doing so. In the 
resolution of the conflict between Germany and France lies, in my 
conviction, the focal point of the entire European war, and" (as stated in 
the Schlieffen memorandum) "the fate of Austria will not be decided on 
the Bug, but on the Seine."
Furthermore, General v. Moltke sought to accommodate the wishes of 
General v. Conrad as much as possible by expressing the hope that it might 
very soon be possible, after the opening of hostilities in the West, to deploy 
additional se

------------------------------------------------------------------
pageno: 020


<pageno>13</pageno>
<header>Moltke's View on the Role of Austria-Hungary.</header>
<body>
The command against Russia, which might become desirable or even necessary,
was dispensable for the first weeks of the war. The extremely delicate
and hardly satisfactorily solvable question of the supreme command in the East
was therefore not addressed in the peace negotiations between the two chiefs of staff.
As far as the position of the allied powers towards each other
seemed possible, the cooperation of the German and Austro-Hungarian
forces for the case of war was regulated. That nevertheless
differences of opinion remained in the basic views
is not surprising given the nature of the alliance relationship:
Thus, General v. Conrad never completely abandoned the idea
of possibly waging a war in the Balkans, regardless
of the considerable weakening of the forces available against Russia
and the resulting additional

------------------------------------------------------------------
pageno: 023


<pageno>16</pageno>
<header>The Armaments and Force Ratios until Summer 1914.</header>
<body>raises several points about the war readiness of the other great powers including Germany, namely: through the elimination of the military weakness period in winter due to retention of the oldest year group until the completed training of recruits, — through frequent practical testing of all mobilization measures with the help of trial and test mobilizations, — through the possibility of extraordinary acceleration of mobilization with the help of the "war preparation period". Based on intelligence about measures during the Balkan Wars of 1912/13, the memorandum further explained that this "war preparation period" allowed, in times of political tension, "to assign supplementary personnel and horses to the troops before the start of actual mobilization" and to prepare these units to such an extent that they could ult

...

------------------------------------------------------------------
pageno: 348


<header>Appendix 1.</header>
<header>Military Formations.</header>
<header>Preliminary Note.</header>
<body>
The military formations reflect the composition of the armies for a specific point in time or period; earlier or later deviations in the composition are indicated. All parts added later are highlighted by special script.
The following abbreviations, which may not be immediately understandable, were used:
A.K. = Army Corps
Abt. = Division
Drag.R. = Dragoon Regiment
Erg./147 = Replacement Battalion Infantry Regiment No. 147
F.A.R. = Field Artillery Regiment
Füs.R. = Fusilier Regiment
G.R. = Guard Regiment
Geb.Brig. = Mountain Brigade
Gr.R. = Grenadier Regiment
Hus.R. = Hussar Regiment
Inf.R. = Infantry Regiment
Jäg.B. = Jäger Battalion
Jäg.R. z. Pf. = Mounted Jäger Regiment
K.R. = Cuirassier Regiment
Ldst.R. = Landsturm
Ldsch. = Landwehr
M.G.A. = Machine Gun Division
Pi.1 = Pioneer Battalion No. 1
1./

------------------------------------------------------------------
pageno: 349


<pageno>358</pageno>
<header>Order of Battle.</header>
<body>
The German 8th Army
on August 26, 1914, 1st day of the Battle of Tannenberg¹).
9 Inf. and Res. Div., 4 Div. Det. and Replacement Troops, 1 Cav. Div. = 158 Batl., 78 Squadrons
140 Batteries (= 774 Guns²).
Strengthened by early September to:
13 Inf. and Res. Div., 5½ Div. Landwehr and Replacement Troops, 2 Cav. Div. = 229 Batl., 119 Squadrons
219 Batteries (= 1194 Guns).
Additionally, Landwehr Corps with 2 Landwehr Div. = 34 Batl., 12 Squadrons, 12 Batteries
(= 72 Guns).
Army High Command 8.
Commander-in-Chief: Generaloberst v. Beneckendorff and v. Hindenburg (until 22. 8. General-
oberst v. Prittwitz and Gaffron)
Chief of Gen. Staff: Genmaj. Ludendorff (until 22. 8. Genmaj. Count v. Waldersee)
1st Gen. Staff Officer: Lt. Col. Hoffmann
Quartermaster General: Genmaj. Grünert
Gen. of Engineers: Genmaj. Resten.
Air Forces.
Field Flying Det. 16
Fortre

------------------------------------------------------------------
pageno: 350
result: The input text is correctly OCR'ed.
------------------------------------------------------------------
pageno: 351
result: The input text is correctly OCR'ed.
------------------------------------------------------------------
pageno: 352
result: The input text is correctly OCR'ed.
------------------------------------------------------------------
pageno: 353


<pageno>362</pageno>
<header>Military Formations.</header>
<body>
Fort Boyen (Lötzen).
4½ Battalions, 1 Squadron, 8 Batteries.
Commander: Colonel Busse
Infantry: Replacement/R.R. 147, IV./Ldw. 18 and 2½ Battalions Ldst.
Machine Guns: 11 Fortress Detachments and Troops
Cavalry: Replacement Squadron Drag. R. 11
Field Artillery: 2 Ldw. Batteries from XX. A.K.
Foot Artillery: 6 unassigned Batteries.
Pioneers: 1 Fortress Battalion Detachment.
Air Forces: 1 XX. Army Corps.
From the Army's Line of Communication Inspection (on 30th and 31st August
call

------------------------------------------------------------------
pageno: 354
result: The input text is correctly OCR'ed.
------------------------------------------------------------------
pageno: 355
result: The input text is correctly OCR'ed.
------------------------------------------------------------------
pageno: 356
result: The input text is correctly OCR'ed.
------------------------------------------------------------------
pageno: 357
result: The input text is correctly OCR'ed.
------------------------------------------------------------------
pageno: 358


<pageno>367</pageno>
<header>The Austro-Hungarian Army.</header>
<body>
2nd Army.
7 Inf. Div., 4 Lfst. Brig., 2 Cav. Div.
= 145 1/4 Batt., 67 3/4 Squadrons, 64 Batteries (= 368 Guns)
Part of the army formed the Kövess Army Group until August 23 (see below).
Army Commander: General of Cavalry v. Böhm-Ermolli ¹)
Chief of Gen. Staff: Genmaj. v. Meenfessy
IV Corps ¹): 31 ¹) and 32 ¹) Inf. Div., 4th March Brig.
VII Corps ¹): 17 

------------------------------------------------------------------
pageno: 359
result: "The input text is correctly OCR'ed."
------------------------------------------------------------------
pageno: 360
result: The input text is correctly OCR'ed.
------------------------------------------------------------------
pageno: 361
result: The input text is correctly OCR'ed.
------------------------------------------------------------------
pageno: 362
result: The input text is correctly OCR'ed.
------------------------------------------------------------------
pageno: 363
result: The input text is correctly OCR'ed.
------------------------------------------------------------------
pageno: 364
result: The input text is correctly OCR'ed.
------------------------------------------------------------------
pageno: 365
result: The input text is correctly OCR'ed.
------------------------------------------------------------------
pageno: 366
result: "The input text is correctly OCR'ed."
------------

In [18]:
from src.utils import dump_fixed_paragraphs_output_to_json

dump_fixed_paragraphs_output_to_json(foldername, english_texts_fixed_paragraphs)

doc2, fname2 = save_document(english_texts_fixed_paragraphs, foldername, language=f'{foldername} - English_fixed_paragraphs')


save_document - L197 - INFO - saved to "../output_data/Der Weltkrieg v2/Der Weltkrieg v2 - English_fixed_paragraphs"
